# Solve the GPE in a 1D parabolic trap
### Ashton Bradley

# Introduction
In this simple example we start by finding the ground state of the Gross-Pitaevskii equation
in a harmonic trap.

The mean field order parameter of a Bose-Einstein condensate far below the critical temperatrure for condensation evolves according to the GP-equation
$$
i\hbar\frac{\partial \psi(x,t)}{\partial t}=\left(-\frac{\hbar^2\partial_x^2}{2m}+V(x,t)+g|\psi(x,t)|^2\right)\psi(x,t)
$$
with potential $V(x,t)=m\omega_x^2 x^2/2$, and positive interaction strength $g$.

We work in harmonic oscillator units, taking length in units of $a_x=\sqrt{\hbar/m\omega_x}$ and time in
units of $1/\omega_x$.

The equation of motion that we solve numerically is
$$
i\frac{\partial \psi(x,t)}{\partial t}=\left(-\frac{\partial_x^2}{2}+\frac{x^2}{2}+g|\psi(x,t)|^2\right)\psi(x,t)
$$
where all quantities are now dimensionless.

# Loading the package
First, we load some useful packages, and set up defaults for `Plots`.

In [ ]:
using Plots, LaTeXStrings
gr(fmt="png",legend=false,titlefontsize=12,size=(500,200),grid=false,transpose=true,colorbar=false);

Now load `FourierGPE`

In [ ]:
using FourierGPE

Let's define a convenient plot function

In [ ]:
function showpsi(x,ψ)
    p1 = plot(x,abs2.(ψ))
    xlabel!(L"x/a_x");ylabel!(L"|\psi|^2")
    p2 = plot(x,angle.(ψ))
    xlabel!(L"x/a_x");ylabel!(L"\textrm{phase}(\psi)")
    p = plot(p1,p2,layout=(2,1),size=(600,400))
    return p
end

Let's set the system size, and number of spatial points and initialize default simulation

In [ ]:
L = (40.0,)
N = (512,)
sim = Sim(L,N)
@unpack_Sim sim;
μ = 25.0

Here we keep most of the default parameters but increase the chemical potential.

## Declaring the potential
Let's define the trapping potential.

In [ ]:
import FourierGPE.V
V(x,t) = 0.5*x^2

We only require the definition as a scalar function
because it will be evaluated on the grid using a broadcasted dot-call.

# Initial condition
Let's define a useful Thomas-Fermi wavefunction

In [ ]:
ψ0(x,μ,g) = sqrt(μ/g)*sqrt(max(1.0-V(x,0.0)/μ,0.0)+im*0.0)
x = X[1];

The initial state is now created on the grid and all modified variables are scooped up into `sim`:

In [ ]:
ψi = ψ0.(x,μ,g)
ϕi = kspace(ψi,sim) #sim uses Fourier transforms that are norm-preserving
@pack_Sim! sim;
sim

The important points to note here are that we have modified $\mu$ and the initial condition $\phi_i$, and we have left the default damping parameter
$\gamma=0.5$ which means we are going to find a ground state of the GPE.

## Default simulation parameters
The source code defining the simulation type `Sim` sets the default values and
also has some further explanation of each variable:

In [ ]:
@with_kw mutable struct Sim{D} <: Simulation{D} @deftype Float64
    # Add more parameters as necessary, or add to params (see examples)
    L::NTuple{D,Float64} # box length scales
    N::NTuple{D,Int64}  # grid points in each dimensions
    μ = 15.0    # chemical potential
    g = 0.1     # interaction parameter
    γ = 0.5; @assert γ >= 0.0 # damping parameter
    ti = 0.0    # initial time
    tf = 2/γ    # final time
    Nt::Int64 = 200     # number of saves over (ti,tf)
    params::UserParams = Params() # optional user parameters
    V0::Array{Float64,D} = zeros(N)
    t::LinRange{Float64} = LinRange(ti,tf,Nt) # time of saves
    ϕi::Array{Complex{Float64},D} = zeros(N) |> complex # initial condition
    alg::OrdinaryDiffEq.OrdinaryDiffEqAdaptiveAlgorithm = Tsit5() # default solver
    reltol::Float64 = 1e-6 # default tolerance; may need to use 1e-7 for corner cases
    flags::UInt32 = FFTW.MEASURE # choose a plan. PATIENT, NO_TIMELIMIT, EXHAUSTIVE
    # === saving
    nfiles::Bool = false
    path::String = nfiles ? joinpath(@__DIR__,"data") : @__DIR__
    filename::String = "save"
    # === arrays, transforms, spectral operators
    X::NTuple{D,Array{Float64,1}} = xvecs(L,N)
    K::NTuple{D,Array{Float64,1}} = kvecs(L,N)
    espec::Array{Complex{Float64},D} = 0.5*k2(K)
    T::TransformLibrary = makeT(X,K,flags=flags)
end

# Evolution in k-space
The `FFTW` library is used to evolve the Gross-Pitaevskii equation in k-space

In [ ]:
sol = runsim(sim);

By default the solver returns all time slices specified by the `t` vector (`t=LinRange(ti,tf,Nt)`) and solution information in a single variable `sol`.

Let's have a look at the final state and verify we have a ground state with the correct chemical potential:

In [ ]:
ϕg = sol[end]
ψg = xspace(ϕg,sim)
p=plot(x,g*abs2.(ψg),fill=(0,0.2),size=(500,200),label=L"gn(x)")
plot!(x,one.(x)*μ,label=L"\mu")
plot!(x,V.(x,0.0),label=L"V(x)",legend=:topright)
xlims!(-10,10); ylims!(0,1.3*μ)
title!(L"\textrm{local}\; \mu(x)")
xlabel!(L"x/a_x"); ylabel!(L"\mu(x)/\hbar\omega_x")
plot(p)

The initial Thomas-Fermi state has been evolved for a default time $t=2/\gamma$ which is
a characteristic damping time for the dissipative system with dimensionless damping
$\gamma$. The solution will approch the ground state satisfying $L\psi_0=\mu\psi_0$ on a timescale of order
$1/\gamma$.

# Dark soliton in harmonically trapped system
We found a ground state by imaginary time propagation.
Now we can impose a phase and density imprint consistent with a dark soliton.
We will use the solution for the homogeneous system, which will be a reasonable
approximation if we impose it on a state that varies slowly over the scale of the soliton (the healing length $\xi$).

## Imprinting a dark soliton

In [ ]:
ψf = xspace(sol[end],sim)
c = sqrt(μ)
ξ = 1/c
v = 0.5*c
xs = 0.
f = sqrt(1-(v/c)^2)

Soliton speed is determined by depth and local healing length, and is intialized at `xs=0.0`.

In [ ]:
ψs = @. ψf*(f*tanh(f*(x-xs)/ξ)+im*v/c)
showpsi(x,ψs)
xlims!(-10,10)

## Initilize Simulation
We can use the previous parameters in `sim` to define a new simulation, while modifying parameters as required (in this case the damping and simulation timescale):

In [ ]:
γ = 0.0
tf = 8*pi/sqrt(2); t = LinRange(ti,tf,Nt)
dt = 0.01π/μ
ϕi = kspace(ψs,sim)
simSoliton = Sim(sim;γ=γ,tf=tf,t=t,ϕi=ϕi) #define a new simulation, using keywords
# @pack_Sim! simSoliton; #we could instead pack everything into simSoliton, since we have made all changes

## Solve equation of motion
As before, we specify the initial condition in momentum space, and evolve

In [ ]:
@time sols = runsim(simSoliton);

## View the solution using Plots
Plots allows easy creation of an animated gif, as in the runnable example code below.

In [ ]:
ϕf = sols[end-4]
ψf = xspace(ϕf,simSoliton)
showpsi(x,ψf)

anim = @animate for i in 1:length(t)-4 #make it periodic by ending early
    ψ = xspace(sols[i],simSoliton)
    y = g*abs2.(ψ)
    p = plot(x,y,fill=(0,0.2),size=(500,200))
    xlims!(-10,10); ylims!(0,1.3*μ)
    title!(L"\textrm{local}\; \mu(x)")
    xlabel!(L"x/a_x"); ylabel!(L"\mu(x)/\hbar\omega_x")
end
animpath = joinpath(@__DIR__,"media/soliton.gif")
gif(anim,animpath,fps=30)

The result is visible in the [media folder](../../media/soliton.gif) of this repository.


Here we simply plot the final state:

In [ ]:
ψ = xspace(sols[end],simSoliton)
y = g*abs2.(ψ)
p=plot(x,y,fill=(0,0.2),size=(500,200))
xlims!(-10,10); ylims!(0,1.3*μ)
title!(L"\textrm{local}\; \mu(x)")
xlabel!(L"x/a_x"); ylabel!(L"\mu(x)/\hbar\omega_x")
plot(p)

The dark soliton executes simple harmonic motion with amplitude detemined by its depth.